In [3]:
predictions = None

import sklearn
import pandas as pd
import numpy as np
import utils
from data import Train_test
from sklearn.pipeline import Pipeline
from custom_transformers import Get_Macro_Data, Clean_Train, Custom_Imputer
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt

utils.create_dirs()

# train and test data paths
train_path = '/content/data/credit_train.csv'
test_path = '/content/data/credit_test.csv'

# training
train_data = pd.read_csv(train_path)


train_test = Train_test(train_data)
train_test.train_test_split()
X_train, X_test, y_train, y_test = train_test.get_train_test_split()


# pipeline to clean up data and grab macro data. iterative imputers may take a while to train
pipeline = Pipeline([('clean', Clean_Train()),
                     ('impute', Custom_Imputer()),
                     ('get macrodata', Get_Macro_Data())])


pipeline.fit(X_train)
X_train = pipeline.transform(X_train)
X_test = pipeline.transform(X_test)


# parameters for xgboost
xgb_params = {'objective': 'binary:logistic', 'use_label_encoder': False, 'base_score': 0.09, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'auc', 'gamma': 0, 'gpu_id': 0, 'grow_policy': 'lossguide', 'importance_type': '', 'interaction_constraints': '', 'learning_rate': 0.2, 'max_bin': 256, 'max_cat_to_onehot': 4, 'max_delta_step': 1, 'max_depth': 1, 'max_leaves': 0, 'min_child_weight': 0.1, 'monotone_constraints': '()', 'n_estimators': 200, 'n_jobs': 4, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 0, 'sampling_method': 'uniform', 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'nthread': 4}

#train xgb
xgboost = XGBClassifier(**xgb_params)
xgboost.fit(X_train, y_train)

y_train_p = xgboost.predict_proba(X_train)[:,1]
yPred_p = xgboost.predict_proba(X_test)[:,1]
print("XGB train roc auc:", roc_auc_score(y_train, y_train_p), "XGB test roc auc:",roc_auc_score(y_test,yPred_p))


# -------Testing-------

test_data = pd.read_csv(test_path)

X_test_new = pipeline.transform(test_data)

probabilities = xgboost.predict_proba(X_test_new)
predictions = probabilities[:,1].tolist() # probability of default



occupation_categories directory already exists


KeyboardInterrupt: ignored